In [11]:
#function of minterm으로 input을 나타내야 하는데, 나는 minterm의 숫자들을 input으로 받도록 하겠다.
#각 숫자들은 그냥 
def Quine_Mccluskey():
    num = int(input('variable의 개수를 넣어주세요: '))
    print('minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요')
    minterm_list=list(map(int,input('function of minterms: f=Σm(').split()))
    
    """본격적인 코드로 들어가기 전에, 이 과정을 아예 시행할 필요가 없는 경우들이 있다. 대표적으로 모든 input이 들어올 때와
    input이 아무것도 들어오지 않을 때는 다음과 같이 minterm_list의 input의 개수만으로 판별할 수 있으므로 이를 먼저 걸러준다.
    이는 알파벳 꼴의 SOP form으로 계산되지 않아 예외 케이스로 반드시 들어가야 한다."""
    
    if len(minterm_list)==2**num: 
        print(1)
        return
    elif minterm_list==[]:
        print(0)
        return
    
    """먼저 input으로 들어온 minterm들을 binary number꼴로 바꿔주어야 한다."""
    binary_number=[]
    for i in range(len(minterm_list)):
        binary_number.append(input_to_binary(minterm_list[i],num))
    
    """이로부터 prime_implicant들을 모두 찾는다. 이에 필요한 함수는 아래 구현해두었다."""
    prime_implicant_list=prime_implicants(binary_number,num) #prime_implicant list생성
    
    #prime implicant chart를 그린다. 이 중 우리가 손으로 그릴 때 X표시 해주었던 부분은 1로 표기할 것이다.
    PI_chart=[[0]*len(binary_number) for _ in range(len(prime_implicant_list))]
    for i in range(len(PI_chart)):
        for j in range(len(PI_chart[0])):
            if numbers_are_same(prime_implicant_list[i],binary_number[j]):
                PI_chart[i][j]=1
    
    #prime implicant chart로부터 essential prime implicant를 찾는다.
    
    epi=essential_prime_implicant(PI_chart) #이 essential_prime_implicant는 index를 반환한다.
    
    not_essential=[]
    essential=[]
    for i in range(len(prime_implicant_list)): #위에 구한 epi리스트로부터 essential prime implicant와 not essential을 구분해준다.
        if i not in epi:
            not_essential.append(prime_implicant_list[i]) #not_essential과 essential을 나눠준다.
        else:
            essential.append(prime_implicant_list[i])
    
    covered=[] #essential prime implicant들에 의해 cover되는 binary_number들을 찾아준다. 이는 petrick's method에서 제외된다.
    for i in range(len(binary_number)):
        for j in range(len(essential)):
            if numbers_are_same(essential[j],binary_number[i]):
                if binary_number[i] not in covered:
                    covered.append(binary_number[i])
                    
    not_covered=[] #essential prime implicant들에 의해 cover되지 않는 binary_number들을 이 배열에 저장해준다.
    for i in range(len(binary_number)):
        if binary_number[i] not in covered:
            not_covered.append(binary_number[i])
    
    """모든 SOP form을 찾을 수 있는 좋은 방법은 petrick's method이다. 이 때 petrick's method를 사용하기 전에 먼저 essential
    prime implicant들을 prime implicant chart로부터 모두 제거해주어야 이들에 의해 cover되는 binary_number도 모두 제거한다.
    이렇게 선별된 prime_implicant_chart를 다시 만들어서 petrick's method를 적용해준다."""
    if len(not_essential)!=0 and len(not_covered)!=0:
        PI_chart2=[[0]*len(not_covered) for _ in range(len(not_essential))]
        for i in range(len(PI_chart2)):
            for j in range(len(PI_chart2[0])):
                if numbers_are_same(not_essential[i],not_covered[j]):
                    PI_chart2[i][j]=1
                    
  
                
        result_of_petrick=petrick_method(PI_chart2)
    else: 
        result_of_petrick=[[]]
        
    """아래에 petrick's method의 결과와 essential prime implicant를 합치는 시행을 했는데, essential prime implicant만으로
    모든 minterm이 cover되는 경우 petrick's method가 돌아가지 않는다. 이를 방지하기 위해 petrick's method를 사용하지 않으면
    result_of_petrick을 비어있는 이중리스트로 설정해주어야 했다."""    
    
    #terms는 petrick's method의 결과로부터 binary_number을 뽑아내는 함수이다.
    terms=[]
    for i in range(len(result_of_petrick)):
        c=[]
        for j in range(len(result_of_petrick[i])):
            r=result_of_petrick[i][j]
            c.append(not_essential[r])
        terms.append(c)
        
    #petrick's method로 가장 term개수가 적은 조합을을 뽑았지만 이들이 정말 최소의 비용으로 만들어졌는지는 알 수 없다.
    #그래서 이 아래는 길이를 비교해서 길이가 최소인 조합들만 terms2로 분류해주는 과정이다.
    length_list=[]
    for i in range(len(terms)):
        total_length=0
        for j in range(len(terms[i])):
            length=0
            for k in range(len(terms[i][j])):
                if terms[i][j][k]!='-':
                    length+=1
            total_length+=length
        length_list.append(total_length)
        
             
    min_length=min(length_list)
    terms2=[]
    for i in range(len(length_list)):
        if length_list[i]==min_length:
            terms2.append(terms[i])
            
    
    #위에 binary_number형태로 바뀐 petrick's method의 결과들에 각각 essential prime implicant들을 합쳐준다.
    for i in range(len(terms2)):
        for j in range(len(essential)):
            terms2[i].append(essential[j])
            
        
    """이제 terms에는 binary형태로 우리가 구하는결과가 들어있다. 이를 minterm으로 바꾸고 더하기 모양으로 출력하는 것이
    우리가 마지막으로 해야 할 일이다."""
    
    #result는 위에 binary꼴로 저장되어있던 결과 리스트를 minterm형태로 바꾸어주는 함수이다.
    result=[]
    for i in range(len(terms2)):
        d=[]
        for j in range(len(terms2[i])):
            d.append(binary_to_minterm(terms2[i][j]))
        result.append(d)
        
    #answer는 minterm형태로 리스트에 담겨있는 결과값을 str type의 SOP form모양으로 바꿔주기 위해 만들었다.    
    answer=[]
    for i in range(len(result)):
        s=''
        for j in range(len(result[i])):
            s=s+result[i][j]
            if j!=len(result[i])-1:
                s=s+'+'
                
        answer.append(s)
    
    #이제 answer리스트에는 우리가 구한 답이 들어있다. 이를 단순히 출력해주기만 하면 된다.
    """아래 for문이 len(result)로 돌아가는 이유는 과정이 복잡해 result에서 answer로 제대로 변환되었는지 확인하기 위해
    저렇게 짰다. 어차피 모든 과정이 정확하게 구동된다면 두 리스트의 길이는 같기 때문에 상관이 없다."""
    print('-----------------------------------Result-------------------------------')
    for i in range(len(result)):
        print(answer[i])
        
    return



"""이 위에는 main으로 돌아가는 함수이다. 아래는 위의 Quine_Mccluskey에서 필요한 method들을 구현한 함수이다.
바로 아래의 input_to_binary는 input으로 들어오는 십진수(minterm)을 이진수로 변환해주는 함수이다."""
def input_to_binary(a,num): #input을 숫자로 넣도록 할 것이다.
    result=[]
    while (a//2)!=0:
        result.append(str(a%2))
        a=a//2
    result.append(str(a))
    if len(result)<num:
        while len(result)!=num:
            result.append('0')
    result.reverse()
    return ''.join(result)


"""----------------------------------------------------------------------------------
이 다음 combine_two_binary와 prime_implicants함수를 통해 prime_implicants를 찾아준다.
-------------------------------------------------------------------------------------"""

"""combine_two_binary는 두 str type의 binary number을 합칠 수 있으면 합친 결과를 반환하고, 합칠 수 없다면
아무것도 반환하지 않는 함수이다."""
def combine_two_binary(a,b,num):
    result = ''
    cnt=0
    for i in range(num):
        if a[i]==b[i]:
            result += a[i]
        elif a[i]!=b[i]:
            result += '-'
            cnt+=1
    if cnt<2:
        return result
    return None

"""아래는 위의 함수를 이용해 prime implicant를 구하는 시행을 코드로 구현한 것이다."""
def prime_implicants(data,num):
    length = len(data)
    combined = []
    checklist = [0]*length
    for i in range(length):
        for j in range(i+1, length):
            c = combine_two_binary(data[i], data[j],num) #prime implicant에서 두 binary number을 합치려는 시행
            if c != None: #만약 두 binary number이 제대로 병합이 된 경우
                combined.append(c) #이를 배열에 저장
                checklist[i] = 1 
                checklist[j] = 1
            else:
                continue


    a_list = []             
    for i in range(len(combined)): #binary_number들을 묶어준 결과들이 들어있는 combined에서 중복되는 implicant를 제거해줌.
        if combined[i] not in a_list:
            a_list.append(combined[i])
            
    
    not_combined = []
    k = 0
    for i in range(length): #시행이 끝나고 묶이지 않은 숫자들을 not_combined에 넣어줌
        if( checklist[i] == 0 ):
            not_combined.append(data[i])
            k = k+1

    if(k == length or length == 1): #input이 0개,1개거나 모든 input이 묶이지 못하는 경우, recursion의 base역할
        return not_combined #묶이지 않는 list를 반환
    
    else:
        return not_combined + prime_implicants(a_list,num) #묶이지 않은 리스트+재귀적으로 묶여있는 애들가지고 한번 더
    
    
def numbers_are_same(prime_num,num): #prime_num이 num을 포함하는 관계에 있거나 둘이 동일할 경우 True를 반환하는 함수임.
    length=len(prime_num)
    for i in range(length):
        if prime_num[i]!='-':
            if prime_num[i]!=num[i]:
                return False
    return True
"""----------------------------------------------------------------------------------
앞서 찾아준 prime implicant를 통해 최적의 해를 구하기 위해 먼저 essential prime implicants
들을 찾아 주어야 한다. essential prime implicants를 구하는 방법에는 여러가지가 있지만 우리는 아래에서
petric's method를 사용해주어야 하기 때문에 나는 손으로 quine-mccluskey를 그릴 때와 동일하게
prime_implicant_chart를 이용해서 이를 구해주겠다.
-------------------------------------------------------------------------------------"""
    
    
def essential_prime_implicant(chart):
    result=[]

    for i in range(len(chart[0])):
        cnt=0
        position=0
        for j in range(len(chart)): #설정을 이렇게 해주면 chart를 세로로 읽을 수 있다.
            if chart[j][i]==1:
                cnt+=1
                position=j

        if cnt==1: #어떤 binary_number이 오직 하나의 prime implicant로만 cover되는 경우
            if position not in result: #이 implicant를 essential prime implicant로 선택함.
                result.append(position)

    return result

"""처음 코드를 짤 때는 prime_implicant_list에 들어있는 값들을 반환하게 했지만 index만 반환하도록 하는 것이
더 간단하다고 생각하여 prime_implicant_list내의 essential_prime_implicants들의 index를 반환하는 함수로 구현하였다."""
            


"""----------------------------------------------------------------------------------
prime implicant chart로부터 SOP solution을 찾아야 하는데, 이 때 우리가 이용할 수 있는 것이
petric method이다. 이 petric_method는 앞서 구현해놓은 PI_chart를 이용해 구한다.
-------------------------------------------------------------------------------------"""

    
def petrick_method(chart):
    set_of_prime=[]
    for i in range(len(chart[0])):
        temp=[]
        for j in range(len(chart)): #설정을 이렇게 해주면 chart를 세로로 읽을 수 있다.
            if chart[j][i]==1:
                temp.append([j])
        set_of_prime.append(temp)
    
    
    for i in range(len(set_of_prime)-1): #petrick's method를 위해 연속적으로 분배법칙을 시행하는 과정임
        a=mul_two_list(set_of_prime[i],set_of_prime[i+1])
        set_of_prime[i+1]=a
        
    #이 때 set_of_prime[-1]에는 petrick's method에서 분배법칙을 마친 결과가 들어있음
    #아래의 과정은 이들 중 최소인(길이가 가장 짧은) 조합을 고르는 과정임
    
    len_of_prime=[]
    for i in range(len(set_of_prime[-1])):
        len_of_prime.append(len(set_of_prime[-1][i]))
    
    m=min(len_of_prime)
    result=[]
    for i in range(len(set_of_prime[-1])): #petrick's method의 결과 중 길이가 짧은 term만 고르는 과정임.
        if len_of_prime[i]==m:
            result.append(set_of_prime[-1][i])
        
    
    return result
    """손으로 petric method를 계산할 때는 포함관계에 있는 term들을 지워주는 시행을 했는데 이 코드에서는 
    굳이 그렇게 할 필요가 없이 길이가 가장 짧은 term을 반환하도록 해도 된다고 생각한다. """
    
def mul_two_list(list1,list2): #분배법칙을 계산하기 위해 만들어준 함수임. petrick's method를 위해 꼭 필요함.
    result=[]
    for i in range(len(list1)):
        for j in range(len(list2)):
            if list1[i]==list2[j]: #두 이중리스트 안에 들어있는 원소가 동일하다면 이를 곱해줘도 동일한 것을 반환하도록
                result.append(list1[i])
            else:
                new_list=list1[i]+list2[j] #두 이중리스트 안에 들어있는 원소가 다르면 둘을 더하도록 했음
                new_list=remove_duplicated(new_list) #중복된 결과가 나오므로 이를 없애줌
                result.append(new_list)

    result=remove_duplicated(result)
    return result


def remove_duplicated(a): #위의 함수를 구현하다가 중복되는 경우를 제거해야해서 이를 위해 구현해준 함수임.
    new_list=[]
    for i in range(len(a)): # not in 을 이용해 중복되는 것들을 제거해준다.
        if a[i] not in new_list:
            new_list.append(a[i])
            
    new_list.sort()
    return new_list
                
"""----------------------------------------------------------------------------------
SOP form으로 반환값을 내놓아야 하기 때문에 기존에 이진수로 계산해 준 값을 boolean꼴로
바꿔주어야 한다. 이를 위해서 아래 함수를 구현하였다.
-------------------------------------------------------------------------------------"""


def binary_to_minterm(num): #마지막에 이진수 리스트를 이걸 돌려서 minterm모양으로 바꿔줄 것이다.
    result=''
    minterms='abcdefghij' #우리는 이 프로그램에서 최대 5자리까지만 고려하기 때문에 사실 e이상으로 쓸일이 없다.
    for i in range(len(num)):
        if num[i]=='1': #number가 1이면 result에 단순히 알파벳을 더해준다.
            result+=minterms[i]
        elif num[i]=='0': #number가 0이면 이 알파벳에 추가로'을 더해준다.
            result=result+minterms[i]+"'"
    return result

In [12]:
for i in range(10):
    Quine_Mccluskey()

variable의 개수를 넣어주세요: 3
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(
0
variable의 개수를 넣어주세요: 3
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(0 1 2 3 4 5 6 7
1
variable의 개수를 넣어주세요: 3
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(0 2 5 6
-----------------------------------Result-------------------------------
ab'c+a'c'+bc'
variable의 개수를 넣어주세요: 4
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(0 1 3 5 6 7 8 10 14 15
-----------------------------------Result-------------------------------
a'b'c'+ab'd'+a'd+bc
b'c'd'+ab'd'+a'd+bc
b'c'd'+acd'+a'd+bc
variable의 개수를 넣어주세요: 4
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(0 3 7 8 9
-----------------------------------Result-------------------------------
b'c'd'+a'cd+ab'c'
variable의 개수를 넣어주세요: 4
minterm의 번호들을 띄어쓰기로 구분해서 넣어주세요
function of minterms: f=Σm(0 1 2 3 4 5 6 7 8 9 10 11 12 13 15
-----------------------------------Result-------------------------------
a'+b'+c'+d
variable의 개수를 넣어주세요: 4
